<a href="https://colab.research.google.com/github/2403a52225-maker/NLP/blob/main/2403A52225_Assignment_6_3(NLP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Dataset-1**

**Load text data**

In [1]:
import pandas as pd
data = pd.read_excel("/content/LDA-Data.xlsx")
print(data.head())

                             News
0   Virat scored century in match
1            BJP won in elections
2  Bumra took 5 wicket in a match
3  Congress form state government


**Text Preprocesing: Clean Text, Word Tokenization, stopword removal, lemma,Rejoin**

In [2]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

def nltk_preprocessing_pipeline(text):
    # Initialize NLTK tools
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    # 1. Preprocess text (from previous steps)
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)

    text = text.lower()  # Convert to lowercase

    # Remove emojis
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE
    )
    text = emoji_pattern.sub(r'', text)

    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces

    # 2. Word Tokenization
    tokenized_words = word_tokenize(text)

    # 3. Stopword Removal
    filtered_words = [word for word in tokenized_words if word not in stop_words]

    # 4. Lemmatization
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

    # 5. Rejoin words
    clean_summary = ' '.join(lemmatized_words)

    return clean_summary

print("NLTK preprocessing pipeline function created successfully!")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


NLTK preprocessing pipeline function created successfully!


In [3]:
data['clean_News'] = data['News'].apply(nltk_preprocessing_pipeline)
print("\nComparison of previous clean_summaries and new clean_summaries_pipeline (first 5 rows):")
print(data[['clean_News']].head())


Comparison of previous clean_summaries and new clean_summaries_pipeline (first 5 rows):
                       clean_News
0      virat scored century match
1                    bjp election
2       bumra took 5 wicket match
3  congress form state government


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
# Vectorize the cleaned summaries
count_vectorizer = CountVectorizer(max_df=0.95, min_df=1, stop_words='english')
doc_term_matrix = count_vectorizer.fit_transform(data['clean_News'])

**BOW**

In [5]:
import pandas as pd
# Get feature (word) names
feature_names = count_vectorizer.get_feature_names_out()
# Convert sparse matrix to DataFrame
bow_df = pd.DataFrame(doc_term_matrix.toarray(),columns=feature_names)
# Display BoW matrix for top 10 documents
bow_top_10 = bow_df.head(10)
print(bow_top_10)

   bjp  bumra  century  congress  election  form  government  match  scored  \
0    0      0        1         0         0     0           0      1       1   
1    1      0        0         0         1     0           0      0       0   
2    0      1        0         0         0     0           0      1       0   
3    0      0        0         1         0     1           1      0       0   

   state  took  virat  wicket  
0      0     0      1       0  
1      0     0      0       0  
2      0     1      0       1  
3      1     0      0       0  


**Apply LDA**

In [6]:
from sklearn.decomposition import LatentDirichletAllocation

# Initialize and fit LDA model
num_topics = 2
LDA = LatentDirichletAllocation(n_components=num_topics, random_state=42)
LDA.fit(doc_term_matrix)

LatentDirichletAllocation(n_components=2, random_state=42)

**Identify words for  each topic**

In [7]:
def display_topics(model, feature_names, num_top_words):
    for topic_idx in range(len(model.components_)):
        print(f"\nTopic {topic_idx}:")

        # Get word weights for this topic
        topic_weights = model.components_[topic_idx]

        # Get indices of words sorted by weight (descending)
        sorted_indices = topic_weights.argsort()[::-1]

        # Take top N words
        top_indices = sorted_indices[:num_top_words]

        # Print top words
        for idx in top_indices:
            print(feature_names[idx], end=" ")
        print()

In [8]:
# Display top words for each topic
num_top_words = 10
print(f"\nTop {num_top_words} words per topic:")
display_topics(LDA, count_vectorizer.get_feature_names_out(), num_top_words)


Top 10 words per topic:

Topic 0:
form government congress state election bjp match wicket bumra took 

Topic 1:
match virat century scored took bumra wicket bjp election state 


**Topic modeling**

In [9]:
# Assign topics to each document
document_topics = LDA.transform(doc_term_matrix)
data['topic'] = document_topics.argmax(axis=1)

print("\nDataFrame with assigned topics (first 5 rows):")
print(data[['clean_News', 'topic']].head())


DataFrame with assigned topics (first 5 rows):
                       clean_News  topic
0      virat scored century match      1
1                    bjp election      0
2       bumra took 5 wicket match      1
3  congress form state government      0


**Dataset 2**

**load text data**

In [10]:
import pandas as pd
data = pd.read_csv("/content/arxiv_data.csv")
print(data.head())

                                              titles  \
0  Survey on Semantic Stereo Matching / Semantic ...   
1  FUTURE-AI: Guiding Principles and Consensus Re...   
2  Enforcing Mutual Consistency of Hard Regions f...   
3  Parameter Decoupling Strategy for Semi-supervi...   
4  Background-Foreground Segmentation for Interio...   

                                           summaries  \
0  Stereo matching is one of the widely used tech...   
1  The recent advancements in artificial intellig...   
2  In this paper, we proposed a novel mutual cons...   
3  Consistency training has proven to be an advan...   
4  To ensure safety in automated driving, the cor...   

                         terms  
0           ['cs.CV', 'cs.LG']  
1  ['cs.CV', 'cs.AI', 'cs.LG']  
2           ['cs.CV', 'cs.AI']  
3                    ['cs.CV']  
4           ['cs.CV', 'cs.LG']  


**Text Preprocesing: Clean Text, Word Tokenization, stopword removal, lemma,Rejoinbold text**

In [11]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

def nltk_preprocessing_pipeline(text):
    # Initialize NLTK tools
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    # 1. Preprocess text (from previous steps)
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)

    text = text.lower()  # Convert to lowercase

    # Remove emojis
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE
    )
    text = emoji_pattern.sub(r'', text)

    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces

    # 2. Word Tokenization
    tokenized_words = word_tokenize(text)

    # 3. Stopword Removal
    filtered_words = [word for word in tokenized_words if word not in stop_words]

    # 4. Lemmatization
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

    # 5. Rejoin words
    clean_summary = ' '.join(lemmatized_words)

    return clean_summary

print("NLTK preprocessing pipeline function created successfully!")

NLTK preprocessing pipeline function created successfully!


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
data['clean_News'] = data['titles'].apply(nltk_preprocessing_pipeline)
print("\nComparison of previous clean_summaries and new clean_summaries_pipeline (first 5 rows):")
print(data[['clean_News']].head())


Comparison of previous clean_summaries and new clean_summaries_pipeline (first 5 rows):
                                          clean_News
0  survey semantic stereo matching semantic depth...
1  futureai guiding principle consensus recommend...
2  enforcing mutual consistency hard region semis...
3  parameter decoupling strategy semisupervised 3...
4  backgroundforeground segmentation interior sen...


In [15]:
from sklearn.feature_extraction.text import CountVectorizer
# Vectorize the cleaned summaries
count_vectorizer = CountVectorizer(max_df=0.95, min_df=1, stop_words='english')
doc_term_matrix = count_vectorizer.fit_transform(data['clean_News'])

**BoW**

In [16]:
import pandas as pd
# Get feature (word) names
feature_names = count_vectorizer.get_feature_names_out()
# Convert sparse matrix to DataFrame
bow_df = pd.DataFrame(doc_term_matrix.toarray(),columns=feature_names)
# Display BoW matrix for top 10 documents
bow_top_10 = bow_df.head(10)
print(bow_top_10)

   01  05mb  09  10  100  1000  10000  100000  1000x  100k  ...  \
0   0     0   0   0    0     0      0       0      0     0  ...   
1   0     0   0   0    0     0      0       0      0     0  ...   
2   0     0   0   0    0     0      0       0      0     0  ...   
3   0     0   0   0    0     0      0       0      0     0  ...   
4   0     0   0   0    0     0      0       0      0     0  ...   
5   0     0   0   0    0     0      0       0      0     0  ...   
6   0     0   0   0    0     0      0       0      0     0  ...   
7   0     0   0   0    0     0      0       0      0     0  ...   
8   0     0   0   0    0     0      0       0      0     0  ...   
9   0     0   0   0    0     0      0       0      0     0  ...   

   zoomtoinpaint  zoonosis  zooplankton  zootuning  zope  zorro  zpd  zscores  \
0              0         0            0          0     0      0    0        0   
1              0         0            0          0     0      0    0        0   
2              0   

**Apply LDA**

In [17]:
from sklearn.decomposition import LatentDirichletAllocation

# Initialize and fit LDA model
num_topics = 2
LDA = LatentDirichletAllocation(n_components=num_topics, random_state=42)
LDA.fit(doc_term_matrix)

LatentDirichletAllocation(n_components=2, random_state=42)

**Identify words for each topic**

In [18]:
def display_topics(model, feature_names, num_top_words):
    for topic_idx in range(len(model.components_)):
        print(f"\nTopic {topic_idx}:")

        # Get word weights for this topic
        topic_weights = model.components_[topic_idx]

        # Get indices of words sorted by weight (descending)
        sorted_indices = topic_weights.argsort()[::-1]

        # Take top N words
        top_indices = sorted_indices[:num_top_words]

        # Print top words
        for idx in top_indices:
            print(feature_names[idx], end=" ")
        print()

In [19]:
# Display top words for each topic
num_top_words = 10
print(f"\nTop {num_top_words} words per topic:")
display_topics(LDA, count_vectorizer.get_feature_names_out(), num_top_words)


Top 10 words per topic:

Topic 0:
learning deep reinforcement representation network using neural image detection time 

Topic 1:
network image graph detection adversarial neural object 3d segmentation using 


**Topic modeling**

In [20]:
# Assign topics to each document
document_topics = LDA.transform(doc_term_matrix)
data['topic'] = document_topics.argmax(axis=1)

print("\nDataFrame with assigned topics (first 5 rows):")
print(data[['clean_News', 'topic']].head())


DataFrame with assigned topics (first 5 rows):
                                          clean_News  topic
0  survey semantic stereo matching semantic depth...      1
1  futureai guiding principle consensus recommend...      0
2  enforcing mutual consistency hard region semis...      1
3  parameter decoupling strategy semisupervised 3...      0
4  backgroundforeground segmentation interior sen...      1
